<h2>Segmenting and Clustering Neighborhoods in Toronto Project</h2>

Step 1 is obtaining postal codes from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

This project will use the BeautifulSoup package to scrape the site and convert the data to a dataframe.

In [1]:
# !pip install beautifulsoup4
# !pip install html5lib

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

Creating a BeatifulSoup object from an url requires the use of the requests package to return the html document.
This is done, and the BeautifulSoup object is imported

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
file=requests.get(url)
text=file.text
soup=bs(text)

We can use BeautifulSoup find_all of the object type 'table' and save them into table. We can then use read_html from Pandas to turn table into a dataframe. This dataframe is a list of all of the tables, in our case the first table is the one we want and we can save it to our desired "neigh" dataframe.

In [4]:
table = soup.find_all('table')
df = pd.read_html(str(table))
neigh=df[0]

In [5]:
neigh.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove the 'not assigned' postal codes 

In [6]:
neigh=neigh[neigh.Borough !='Not assigned']

and combining identical postal codes

In [7]:
neigh=neigh.groupby(['Postcode','Borough'], as_index=False).agg({'Neighbourhood':lambda x: ', '.join(x)})

and rename any un-named Neighourboods

In [8]:
neigh.loc[(neigh.Neighbourhood=='Not assigned'),'Neighbourhood']=neigh.loc[(neigh.Neighbourhood=='Not assigned'),'Borough']
neigh.shape

(103, 3)

In [9]:
#! pip install geocoder

In [10]:
import geocoder

Creating a new data frame and adding columns for latitude and longitude

In [11]:
neigh_latlng=neigh
neigh_latlng['Latitude']=""
neigh_latlng['Longitude']=""

I couldn't get the google geocoder service to work right, so I utilized arcgis, it is a bit slow, but works fine.

In [12]:
for label,row in neigh_latlng.iterrows():
    neigh_latlng.loc[label,'Latitude'] = geocoder.arcgis(row['Postcode']).latlng[0]
    neigh_latlng.loc[label,'Longitude'] = geocoder.arcgis(row['Postcode']).latlng[1]

In [13]:
 neigh_latlng.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8115,-79.1955
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7857,-79.1587
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7658,-79.1752
3,M1G,Scarborough,Woburn,43.7684,-79.2176
4,M1H,Scarborough,Cedarbrae,43.7697,-79.2394
